In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from fastai.text import *

In [3]:
#torch.cuda.empty_cache()

In [4]:
bs = 64

In [5]:
path = untar_data(URLs.HUMAN_NUMBERS)
path.ls()

[PosixPath('/home/ricky/.fastai/data/human_numbers/train.txt'),
 PosixPath('/home/ricky/.fastai/data/human_numbers/valid.txt')]

In [6]:
def readnums(d):
    return [', '.join(o.strip() for o in open(path/d).readlines())]

In [7]:
!head -n 5 {path}/train.txt

one 
two 
three 
four 
five 


In [8]:
!head -n 5 {path}/valid.txt

eight thousand one 
eight thousand two 
eight thousand three 
eight thousand four 
eight thousand five 


In [9]:
train_txt = readnums('train.txt'); train_txt[0][:80]

'one, two, three, four, five, six, seven, eight, nine, ten, eleven, twelve, thirt'

In [10]:
valid_txt = readnums('valid.txt'); valid_txt[0][-80:]

' nine thousand nine hundred ninety eight, nine thousand nine hundred ninety nine'

In [11]:
train = TextList(train_txt, path=path)
valid = TextList(valid_txt, path=path)

src = ItemLists(path=path, train=train, valid=valid).label_for_lm()
data = src.databunch(bs=bs)

In [12]:
train[0].text[0], train[0].text[1], train[0].text[2]

('x', 'x', 'b')

In [13]:
train[0].text[:3]

'xxb'

In [14]:
train[0].text[:80]

'xxbos one , two , three , four , five , six , seven , eight , nine , ten , eleve'

In [15]:
len(data.valid_ds[0][0].data)

13017

In [16]:
data.bptt, len(data.valid_dl)

(70, 3)

13017 tokens, with about ~70 tokens in about a line of text, and 64 lines of text per batch

In [17]:
13017/70/bs

2.905580357142857

In [18]:
it = iter(data.valid_dl)
x1, y1 = next(it)

In [19]:
x1.shape

torch.Size([64, 70])

In [20]:
x1

tensor([[ 2, 19, 11,  ..., 36,  9, 19],
        [ 9, 19, 11,  ..., 24, 20,  9],
        [11, 27, 18,  ...,  9, 19, 11],
        ...,
        [20, 11, 20,  ..., 11, 20, 10],
        [20, 11, 20,  ..., 24,  9, 20],
        [20, 10, 26,  ..., 20, 11, 20]], device='cuda:0')

In [22]:
y1

tensor([[19, 11, 12,  ...,  9, 19, 11],
        [19, 11, 23,  ..., 20,  9, 19],
        [27, 18,  9,  ..., 19, 11, 12],
        ...,
        [11, 20, 10,  ..., 20, 10, 21],
        [11, 20, 10,  ...,  9, 20, 11],
        [10, 26,  9,  ..., 11, 20, 10]], device='cuda:0')

In [23]:
it = iter(data.valid_dl)
x1, y1 = next(it)
x2, y2 = next(it)
x3, y3 = next(it)
it.close()

In [24]:
x1

tensor([[11, 37,  9,  ..., 22, 13,  9],
        [19, 11, 25,  ...,  9, 19, 11],
        [12, 10, 12,  ..., 10, 31,  9],
        ...,
        [21, 12,  9,  ..., 20, 10, 22],
        [11, 20, 10,  ...,  9, 20, 11],
        [10, 27,  9,  ..., 11, 20, 10]], device='cuda:0')

In [25]:
x1.shape, x2.shape, x3.shape

(torch.Size([64, 70]), torch.Size([64, 70]), torch.Size([64, 70]))

In [26]:
x1.numel() + x2.numel() + x3.numel()

13440

In [27]:
x1.shape, y1.shape

(torch.Size([64, 70]), torch.Size([64, 70]))

In [28]:
x2.shape, y2.shape

(torch.Size([64, 70]), torch.Size([64, 70]))

In [29]:
x3.shape, y2.shape

(torch.Size([64, 70]), torch.Size([64, 70]))

In [40]:
v = data.valid_ds.vocab

In [41]:
v.itos

['xxunk',
 'xxpad',
 'xxbos',
 'xxeos',
 'xxfld',
 'xxmaj',
 'xxup',
 'xxrep',
 'xxwrep',
 ',',
 'hundred',
 'thousand',
 'one',
 'two',
 'three',
 'four',
 'five',
 'six',
 'seven',
 'eight',
 'nine',
 'twenty',
 'thirty',
 'forty',
 'fifty',
 'sixty',
 'seventy',
 'eighty',
 'ninety',
 'ten',
 'eleven',
 'twelve',
 'thirteen',
 'fourteen',
 'fifteen',
 'sixteen',
 'seventeen',
 'eighteen',
 'nineteen',
 'xxfake']

In [42]:
x1[:,0]

tensor([11, 19, 12, 10,  9, 12, 21, 19, 27, 19, 11, 15,  9, 10,  9, 15, 21, 19,
        27, 10, 11, 18, 18, 18,  9, 18, 22, 19, 10,  9, 20,  9,  9, 20, 11, 11,
        13, 11, 13,  9, 13,  9, 20, 10, 15, 11, 25, 11, 16,  9, 16, 11, 20, 10,
        11, 11, 25, 11, 19,  9, 19, 21, 11, 10], device='cuda:0')

In [43]:
y1[:,0]

tensor([37, 11, 10, 22, 19, 10, 15, 11, 14, 11, 14,  9, 19, 22, 19, 10, 18, 11,
        17, 37, 17,  9, 10, 10, 19, 10,  9, 11, 27, 19, 10, 19, 20,  9, 12, 12,
         9, 12, 10, 20, 10, 20, 11, 26,  9, 15, 16, 15, 10, 20, 10, 17, 11, 26,
        18, 18, 19, 18, 10, 20, 10, 12, 20, 27], device='cuda:0')

In [44]:
x1

tensor([[11, 37,  9,  ..., 22, 13,  9],
        [19, 11, 25,  ...,  9, 19, 11],
        [12, 10, 12,  ..., 10, 31,  9],
        ...,
        [21, 12,  9,  ..., 20, 10, 22],
        [11, 20, 10,  ...,  9, 20, 11],
        [10, 27,  9,  ..., 11, 20, 10]], device='cuda:0')

In [45]:
y1[:,0]

tensor([37, 11, 10, 22, 19, 10, 15, 11, 14, 11, 14,  9, 19, 22, 19, 10, 18, 11,
        17, 37, 17,  9, 10, 10, 19, 10,  9, 11, 27, 19, 10, 19, 20,  9, 12, 12,
         9, 12, 10, 20, 10, 20, 11, 26,  9, 15, 16, 15, 10, 20, 10, 17, 11, 26,
        18, 18, 19, 18, 10, 20, 10, 12, 20, 27], device='cuda:0')

In [46]:
v.itos[9], v.itos[11], v.itos[12], v.itos[13], v.itos[10]

(',', 'thousand', 'one', 'two', 'hundred')

In [47]:
v.textify(x1[0])

'thousand eighteen , eight thousand nineteen , eight thousand twenty , eight thousand twenty one , eight thousand twenty two , eight thousand twenty three , eight thousand twenty four , eight thousand twenty five , eight thousand twenty six , eight thousand twenty seven , eight thousand twenty eight , eight thousand twenty nine , eight thousand thirty , eight thousand thirty one , eight thousand thirty two ,'

In [48]:
v.textify(x1[1])

'eight thousand sixty , eight thousand sixty one , eight thousand sixty two , eight thousand sixty three , eight thousand sixty four , eight thousand sixty five , eight thousand sixty six , eight thousand sixty seven , eight thousand sixty eight , eight thousand sixty nine , eight thousand seventy , eight thousand seventy one , eight thousand seventy two , eight thousand seventy three , eight thousand'

In [49]:
??v.textify

In [50]:
v.textify(x1[0])

'thousand eighteen , eight thousand nineteen , eight thousand twenty , eight thousand twenty one , eight thousand twenty two , eight thousand twenty three , eight thousand twenty four , eight thousand twenty five , eight thousand twenty six , eight thousand twenty seven , eight thousand twenty eight , eight thousand twenty nine , eight thousand thirty , eight thousand thirty one , eight thousand thirty two ,'

In [51]:
v.textify(x2[1])

'seventy four , eight thousand seventy five , eight thousand seventy six , eight thousand seventy seven , eight thousand seventy eight , eight thousand seventy nine , eight thousand eighty , eight thousand eighty one , eight thousand eighty two , eight thousand eighty three , eight thousand eighty four , eight thousand eighty five , eight thousand eighty six , eight thousand eighty seven , eight thousand eighty'

In [52]:
v.textify(y1[0])

'eighteen , eight thousand nineteen , eight thousand twenty , eight thousand twenty one , eight thousand twenty two , eight thousand twenty three , eight thousand twenty four , eight thousand twenty five , eight thousand twenty six , eight thousand twenty seven , eight thousand twenty eight , eight thousand twenty nine , eight thousand thirty , eight thousand thirty one , eight thousand thirty two , eight'

In [53]:
v.textify(x2[0])

'eight thousand thirty three , eight thousand thirty four , eight thousand thirty five , eight thousand thirty six , eight thousand thirty seven , eight thousand thirty eight , eight thousand thirty nine , eight thousand forty , eight thousand forty one , eight thousand forty two , eight thousand forty three , eight thousand forty four , eight thousand forty five , eight thousand forty six , eight'

In [55]:
v.textify(x3[0])

'thousand forty seven , eight thousand forty eight , eight thousand forty nine , eight thousand fifty , eight thousand fifty one , eight thousand fifty two , eight thousand fifty three , eight thousand fifty four , eight thousand fifty five , eight thousand fifty six , eight thousand fifty seven , eight thousand fifty eight , eight thousand fifty nine , eight thousand sixty , eight thousand sixty'

In [56]:
v.textify(x1[1])

'eight thousand sixty , eight thousand sixty one , eight thousand sixty two , eight thousand sixty three , eight thousand sixty four , eight thousand sixty five , eight thousand sixty six , eight thousand sixty seven , eight thousand sixty eight , eight thousand sixty nine , eight thousand seventy , eight thousand seventy one , eight thousand seventy two , eight thousand seventy three , eight thousand'

In [57]:
v.textify(x2[1])

'seventy four , eight thousand seventy five , eight thousand seventy six , eight thousand seventy seven , eight thousand seventy eight , eight thousand seventy nine , eight thousand eighty , eight thousand eighty one , eight thousand eighty two , eight thousand eighty three , eight thousand eighty four , eight thousand eighty five , eight thousand eighty six , eight thousand eighty seven , eight thousand eighty'

In [58]:
v.textify(x3[1])

'eight , eight thousand eighty nine , eight thousand ninety , eight thousand ninety one , eight thousand ninety two , eight thousand ninety three , eight thousand ninety four , eight thousand ninety five , eight thousand ninety six , eight thousand ninety seven , eight thousand ninety eight , eight thousand ninety nine , eight thousand one hundred , eight thousand one hundred one , eight thousand one'

In [59]:
v.textify(x3[-1])

'thousand two , eight thousand three , eight thousand four , eight thousand five , eight thousand six , eight thousand seven , eight thousand eight , eight thousand nine , eight thousand ten , eight thousand eleven , eight thousand twelve , eight thousand thirteen , eight thousand fourteen , eight thousand fifteen , eight thousand sixteen , eight thousand seventeen , eight thousand eighteen , eight thousand nineteen'

In [60]:
data.show_batch(ds_type=DatasetType.Valid)

idx,text
0,"one , eight thousand sixty two , eight thousand sixty three , eight thousand sixty four , eight thousand sixty five , eight thousand sixty six , eight thousand sixty seven , eight thousand sixty eight , eight thousand sixty nine , eight thousand seventy , eight thousand seventy one , eight thousand seventy two , eight thousand seventy three , eight thousand seventy four , eight thousand seventy five"
1,"hundred two , eight thousand one hundred three , eight thousand one hundred four , eight thousand one hundred five , eight thousand one hundred six , eight thousand one hundred seven , eight thousand one hundred eight , eight thousand one hundred nine , eight thousand one hundred ten , eight thousand one hundred eleven , eight thousand one hundred twelve , eight thousand one hundred thirteen , eight"
2,"one hundred thirty four , eight thousand one hundred thirty five , eight thousand one hundred thirty six , eight thousand one hundred thirty seven , eight thousand one hundred thirty eight , eight thousand one hundred thirty nine , eight thousand one hundred forty , eight thousand one hundred forty one , eight thousand one hundred forty two , eight thousand one hundred forty three , eight thousand one"
3,", eight thousand one hundred sixty four , eight thousand one hundred sixty five , eight thousand one hundred sixty six , eight thousand one hundred sixty seven , eight thousand one hundred sixty eight , eight thousand one hundred sixty nine , eight thousand one hundred seventy , eight thousand one hundred seventy one , eight thousand one hundred seventy two , eight thousand one hundred seventy three ,"
4,"one hundred ninety three , eight thousand one hundred ninety four , eight thousand one hundred ninety five , eight thousand one hundred ninety six , eight thousand one hundred ninety seven , eight thousand one hundred ninety eight , eight thousand one hundred ninety nine , eight thousand two hundred , eight thousand two hundred one , eight thousand two hundred two , eight thousand two hundred three ,"


## Single fully connected model 

In [61]:
data = src.databunch(bs=bs, bptt=3)

In [62]:
x,y = data.one_batch()
x.shape, y.shape

(torch.Size([64, 3]), torch.Size([64, 3]))

In [63]:
nv = len(v.itos); nv

40

In [64]:
nh=64

In [65]:
def loss4(input, target):
    return F.cross_entropy(input, target[:,-1])

def acc4(input, target):
    return accuracy(input, target[:,-1])

In [66]:
??accuracy

In [67]:
x[:,0]

tensor([13, 13, 10,  9, 18,  9, 11, 11, 13, 19, 16, 23, 24,  9, 12,  9, 13, 14,
        15, 11, 10, 22, 15,  9, 10, 14, 11, 16, 10, 28, 11,  9, 20,  9, 15, 15,
        11, 18, 10, 28, 23, 24,  9, 16, 10, 16, 19, 20, 12, 10, 22, 16, 17, 17,
        17, 11, 24, 10,  9, 15, 16,  9, 18, 11])

In [72]:
e = nn.Embedding(2, 4)
e(torch.LongTensor([1]))

tensor([[ 0.9053, -0.7491,  0.5140,  1.8174]], grad_fn=<EmbeddingBackward>)

In [73]:
e(torch.LongTensor([0]))

tensor([[ 1.1923,  1.5305, -0.2876,  0.7356]], grad_fn=<EmbeddingBackward>)

In [74]:
e(torch.LongTensor([1,0]))

tensor([[ 0.9053, -0.7491,  0.5140,  1.8174],
        [ 1.1923,  1.5305, -0.2876,  0.7356]], grad_fn=<EmbeddingBackward>)

In [75]:
e(torch.LongTensor([0,1]))

tensor([[ 1.1923,  1.5305, -0.2876,  0.7356],
        [ 0.9053, -0.7491,  0.5140,  1.8174]], grad_fn=<EmbeddingBackward>)

In [76]:
class Model0(nn.Module):
    def __init__(self):
        super().__init__()
        self.i_h = nn.Embedding(nv,nh)  # green arrow
        self.h_h = nn.Linear(nh,nh)     # brown arrow
        self.h_o = nn.Linear(nh,nv)     # blue arrow
        self.bn = nn.BatchNorm1d(nh)
        
    def forward(self, x):
        h = self.bn(F.relu(self.i_h(x[:,0])))
        if x.shape[1]>1:
            h = h + self.i_h(x[:,1])
            h = self.bn(F.relu(self.h_h(h)))
        if x.shape[1]>2:
            h = h + self.i_h(x[:,2])
            h = self.bn(F.relu(self.h_h(h)))
        return self.h_o(h)

In [129]:
m = Model0()
m.to(x.device)
m(x).shape

torch.Size([64, 40])

In [77]:
learn = Learner(data, Model0(), loss_func=loss4, metrics=acc4)

In [78]:
learn.fit_one_cycle(6, 1e-4)

epoch,train_loss,valid_loss,acc4,time
0,3.598087,3.523987,0.062960,00:00
1,2.711023,2.832337,0.419577,00:00
2,2.120992,2.372223,0.432215,00:00
3,1.897061,2.189157,0.455193,00:00
4,1.819713,2.133675,0.463695,00:00
5,1.803781,2.126478,0.464154,00:00


## Same thing with a loop 

In [81]:
l = torch.LongTensor([1,3])
l.device

device(type='cpu')

In [82]:
class Model1(nn.Module):
    def __init__(self):
        super().__init__()
        self.i_h = nn.Embedding(nv,nh)  # green arrow
        self.h_h = nn.Linear(nh,nh)     # brown arrow
        self.h_o = nn.Linear(nh,nv)     # blue arrow
        self.bn = nn.BatchNorm1d(nh)
        
    def forward(self, x):
        h = torch.zeros(x.shape[0], nh).to(device=x.device)
        for i in range(x.shape[1]):
            h = h + self.i_h(x[:,i])
            h = self.bn(F.relu(self.h_h(h)))
        return self.h_o(h)

In [83]:
learn = Learner(data, Model1(), loss_func=loss4, metrics=acc4)

In [84]:
learn.fit_one_cycle(6, 1e-4)

epoch,train_loss,valid_loss,acc4,time
0,3.558175,3.544861,0.081801,00:00
1,2.662866,2.625846,0.426700,00:00
2,2.083312,2.179972,0.457261,00:00
3,1.875386,2.046385,0.460938,00:00
4,1.805160,2.008627,0.469210,00:00
5,1.790406,2.003796,0.469210,00:00


## Multi fully connected model 

In [85]:
data = src.databunch(bs=bs, bptt=20)

In [86]:
x, y = data.one_batch()
x.shape, y.shape

(torch.Size([64, 20]), torch.Size([64, 20]))

In [88]:
class Model2(nn.Module):
    def __init__(self):
        super().__init__()
        self.i_h = nn.Embedding(nv,nh)
        self.h_h = nn.Linear(nh,nh)
        self.h_o = nn.Linear(nh,nv)
        self.bn = nn.BatchNorm1d(nh)
        
    def forward(self, x):
        h = torch.zeros(x.shape[0], nh).to(device=x.device)
        res = []
        for i in range(x.shape[1]):
            h = h + self.i_h(x[:,i])
            h = F.relu(self.h_h(h))
            res.append(self.h_o(self.bn(h)))
        return torch.stack(res, dim=1)

In [91]:
m = Model2()
m(x).shape

torch.Size([64, 20, 40])

In [92]:
learn = Learner(data, Model2(), metrics=accuracy)

In [93]:
learn.fit_one_cycle(10, 1e-4, pct_start=0.1)

epoch,train_loss,valid_loss,accuracy,time
0,3.806533,3.648143,0.065128,00:00
1,3.610948,3.417284,0.150355,00:00
2,3.376051,3.202450,0.226420,00:00
3,3.148839,3.025108,0.272088,00:00
4,2.955103,2.897585,0.343182,00:00
5,2.804207,2.812480,0.375284,00:00
6,2.695646,2.760278,0.386009,00:00
7,2.623472,2.732453,0.390270,00:00
8,2.580254,2.721726,0.393963,00:00
9,2.557910,2.720105,0.394318,00:00


### Maintain state 

In [98]:
class Model3(nn.Module):
    def __init__(self):
        super().__init__()
        self.i_h = nn.Embedding(nv,nh)
        self.h_h = nn.Linear(nh,nh)
        self.h_o = nn.Linear(nh,nv)
        self.bn = nn.BatchNorm1d(nh)
        self.h = torch.zeros(bs, nh).cuda()
        
    def forward(self, x):
        res = []
        h = self.h
        for i in range(x.shape[1]):
            h = h + self.i_h(x[:,i])
            h = F.relu(self.h_h(h))
            res.append(self.bn(h))
        self.h = h.detach()
        res = torch.stack(res, dim=1)
        res = self.h_o(res)
        return res

In [125]:
m = Model3()
m.to(x.device)
m.h

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')

In [127]:
m(x).shape

torch.Size([64, 20, 40])

In [100]:
learn = Learner(data, Model3(), metrics=accuracy)

In [101]:
learn.fit_one_cycle(20, 3e-3)

epoch,train_loss,valid_loss,accuracy,time
0,3.614492,3.581957,0.083097,00:00
1,3.079314,2.878749,0.295881,00:00
2,2.341001,2.180111,0.331321,00:00
3,1.875286,2.104825,0.315696,00:00
4,1.650962,2.177880,0.318253,00:00
5,1.535159,2.108904,0.319531,00:00
6,1.447931,1.931866,0.363849,00:00
7,1.331122,1.815134,0.466832,00:00
8,1.163093,1.696776,0.507955,00:00
9,0.995415,1.707811,0.520028,00:00


### nn.RNN 

In [104]:
class Model4(nn.Module):
    def __init__(self):
        super().__init__()
        self.i_h = nn.Embedding(nv,nh)
        self.rnn = nn.RNN(nh,nh, batch_first=True)
        self.h_o = nn.Linear(nh,nv)
        self.bn = BatchNorm1dFlat(nh)
        self.h = torch.zeros(1, bs, nh).cuda()
        
    def forward(self, x):
        res,h = self.rnn(self.i_h(x), self.h)
        self.h = h.detach()
        return self.h_o(self.bn(res))

In [112]:
learn = Learner(data, Model4(), metrics=accuracy)

In [113]:
learn.fit_one_cycle(20, 3e-3)

epoch,train_loss,valid_loss,accuracy,time
0,3.434440,3.436819,0.162074,00:00
1,2.758440,2.280135,0.462145,00:00
2,2.113796,2.051751,0.323224,00:00
3,1.762656,2.139951,0.315980,00:00
4,1.575136,1.896961,0.416193,00:00
5,1.436887,1.836449,0.403409,00:00
6,1.269006,1.751699,0.426492,00:00
7,1.104291,1.731689,0.423651,00:00
8,0.944870,1.696497,0.454830,00:00
9,0.798782,1.740499,0.460298,00:00


In [120]:
m = Model4()
m.to(x.device)

Model4(
  (i_h): Embedding(40, 64)
  (rnn): RNN(64, 64, batch_first=True)
  (h_o): Linear(in_features=64, out_features=40, bias=True)
  (bn): BatchNorm1dFlat(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)

In [122]:
x, y = next(iter(data.valid_dl))
m(x).shape

torch.Size([64, 20, 40])

### 2-layer GRU 

In [130]:
class Model5(nn.Module):
    def __init__(self):
        super().__init__()
        self.i_h = nn.Embedding(nv,nh)
        self.rnn = nn.GRU(nh, nh, 2, batch_first=True)
        self.h_o = nn.Linear(nh,nv)
        self.bn = BatchNorm1dFlat(nh)
        self.h = torch.zeros(2, bs, nh).cuda()
        
    def forward(self, x):
        res,h = self.rnn(self.i_h(x), self.h)
        self.h = h.detach()
        return self.h_o(self.bn(res))

In [133]:
learn = Learner(data, Model5(), metrics=accuracy)

In [134]:
learn.fit_one_cycle(10, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,2.624176,2.443607,0.342330,00:00
1,1.569449,1.365696,0.660014,00:00
2,0.760091,1.000079,0.805256,00:00
3,0.365899,0.858224,0.840483,00:00
4,0.185383,1.004048,0.826705,00:00
5,0.098987,1.080673,0.817614,00:00
6,0.056059,1.029932,0.823153,00:00
7,0.033756,0.986360,0.819460,00:00
8,0.022080,1.102736,0.817187,00:00
9,0.015907,1.089085,0.814347,00:00
